In [1]:
%matplotlib qt4
from __future__ import division

from models import tools, filters

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict

sns.set_style("ticks", {"legend.frameon": True})

/home/pavel/.pyenv/al/local/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
data = tools.load_data(limit=3000000)
data = data[data['response_time'] < 15000]

Loaded 3000000 answers.


/home/pavel/.pyenv/al/local/lib/python2.7/site-packages/pandas/io/parsers.py:1159: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
data = data[filters.countries(data)]

In [4]:
_, bins = pd.cut(data['response_time'], bins=20, retbins=True)
intervals = zip(bins[:-1], bins[1:])

responses = defaultdict(lambda: [])
for lower_bound, upper_bound in intervals:
    tools.echo('{}-{}'.format(lower_bound, upper_bound))
    for place in data['place_id'].unique():
        vals = data[(data['response_time'] >= lower_bound) &
                    (data['response_time'] < upper_bound) &
                    (data['place_id'] == place)]
        responses[place].append(vals['is_correct'].mean())

14249.85-14999.0


In [5]:
X = [[] for _ in intervals]
for place in responses:
    for i, value in enumerate(responses[place]):
        if np.isfinite(value):
            X[i].append(value)

In [7]:
labels = ['({}, {}]'.format(int(i), int(j)) for i, j in intervals]

plt.figure(num=None, figsize=(9, 6), dpi=120)
plt.xticks(rotation=70)
bp = plt.boxplot(X, labels=labels, showfliers=False)
plt.xlabel('Response time in miliseconds')
plt.ylabel('Probability of recall')
plt.subplots_adjust(bottom=0.25)

plt.setp(bp['medians'], color='orange')
plt.setp(bp['boxes'], color='#02A5F4')
plt.setp(bp['whiskers'], color='#02A5F4')
plt.setp(bp['fliers'], color='#02A5F4', marker='+')

plt.tight_layout()